Übung: 3a Overlay Netzwerk
------------------------------------------

![](https://kubernetes.io/docs/images/kubernetes-cluster-network.svg)

Quelle: Kubernetes
- - -

Ein Kubernetes Overlay Network ist eine Netzwerktechnologie, die es ermöglicht, Container in einem Kubernetes-Cluster über verschiedene physische oder virtuelle Netzwerke hinweg zu verbinden. 

Diese Netzwerke abstrahieren die physische Netzwerkstruktur und bieten eine einheitliche Kommunikationsschicht für die Pods innerhalb des Clusters.

- - - 

### Nodes (nodes)

Zuerst schauen wir uns das Netzwerk, inkl. der IP-Adressen, der Kubernetes Nodes an:

In [ ]:
! kubectl get nodes -o custom-columns=NAME:.metadata.name,STATUS:.status.conditions[-1].type,IPS:.status.addresses[*].address,OS:.status.nodeInfo.osImage

### Subnets pro Nodes (pods)

In einem Kubernetes-Cluster hat jede Node in der Regel ein eigenes IP-Subnetz, um die Netzwerkkommunikation zu optimieren und zu isolieren. 

Folgende Ausgabe mit Pod Name, IP-Adresse und ausführende Node macht dies sichtbar

In [ ]:
! kubectl get pods --all-namespaces -o custom-columns=NAME:.metadata.name,IP:.status.podIP,NODE:.spec.nodeName | sort -k3

### Service Netzwerk - Node übergreifend (services)

Das Service-Netzwerk ermöglicht die nahtlose Interaktion zwischen verschiedenen Komponenten innerhalb und ausserhalb des Clusters. 

Ein Service in Kubernetes ist eine Abstraktion, die eine Gruppe von Pods über ein Netzwerk zugänglich macht.

Die Services in Kubernetes sind in einem Subnet zusammengefasst:

In [ ]:
! kubectl get services --all-namespaces -o custom-columns=NAME:.metadata.name,IP:.spec.clusterIP,PORT:.spec.ports[*].port

### Ein Pod und zwei Container

Innerhalb eines Pods teilen sich die Container dieselbe IP-Adresse.

Ein Pod mit zwei Containern, die beide auf Port 8080 lauschen, wird fehlschlagen:

In [ ]:
%%bash
cat <<EOF | kubectl apply -f -
apiVersion: v1
kind: Pod
metadata:
  name: webshop-fail
spec:
  containers:
  - name: catalog
    image: registry.gitlab.com/ch-mc-b/autoshop-ms/app/shop/catalog:2.1.0
    ports:
    - containerPort: 8080    
  - name: order
    image: registry.gitlab.com/ch-mc-b/autoshop-ms/app/shop/order:2.1.0
    ports:
    - containerPort: 8080    
EOF
echo "------------- Logs ------------------"
kubectl logs pod/webshop-fail -c catalog
kubectl logs pod/webshop-fail -c order

In [ ]:
! kubectl delete pod/webshop-fail